# Convex Optimization Problem: Traveling Salesman Problem (TSP)
- The objective is to minimize the total distance a salesperson must travel to visit all points (cities) exactly once and return to the starting point. This is done using integer programming techniques with binary decision variables.
- The cost function for this Traveling Salesman Problem (TSP) is designed to minimize the total distance traveled by the salesperson. This is represented as the sum of the distances between each pair of points (cities) that the salesperson chooses to visit consecutively.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp
from geopy import distance

Defining the coordinates of seven points (O, A, B, C, D, E, F) in the format (latitude, longitude).

In [2]:
# Original Data
points = [(-12.059296, -76.975893), #origin O
          (-12.079575, -77.009686), #Point A
          (-12.087303, -76.996620), #Point B
          (-12.084391, -76.975651), #Point C
          (-12.063603, -76.960483), #Point D
          (-12.056762, -77.014452), #Point E
          (-12.011531, -77.002383)] #Point F

The matrix C stores the distances between every pair of points (cities). These distances are computed using the geopy.distance function, which calculates the geographical distance (in kilometers) between two points given their latitude and longitude.

In [3]:
# Building distance matrix
n = len(points)
#Initializing the distance matrix C of size n x n (where n is the number of points
C = np.zeros((n,n))
for i in range(0, n):
    for j in range(0, len(points)):
        #Filling matrix with the pairwise distances between all points using the geopy.distance function.
        C[i,j] = distance.distance(points[i], points[j]).km

# Showing distance matrix
print('Distance Matrix is:\n')
print(np.round(C,4))

Distance Matrix is:

[[0.     4.3092 3.8329 2.7762 1.7441 4.2076 6.0199]
 [4.3092 0.     1.6596 3.7435 5.6408 2.5764 7.5691]
 [3.8329 1.6596 0.     2.3055 4.7278 3.8966 8.4056]
 [2.7762 3.7435 2.3055 0.     2.8311 5.2142 8.5694]
 [1.7441 5.6408 4.7278 2.8311 0.     5.9246 7.3483]
 [4.2076 2.5764 3.8966 5.2142 5.9246 0.     5.1733]
 [6.0199 7.5691 8.4056 8.5694 7.3483 5.1733 0.    ]]


X: This is a binary decision variable matrix of shape n x n, where:
- X[i,j]=1 if the salesperson travels directly from point i to point j.
- X[i,j]=0 otherwise.
- The decision variables represent whether or not a specific connection between two points is chosen.

u: This is an integer variable used to eliminate subtours (subsequences of nodes that are visited separately from the main tour). The u variables represent the order in which the points are visited.

In [4]:
# Defining variables
#Defining the decision variable X, a binary matrix that represents whether there is a connection between points i and j.
X = cp.Variable(C.shape, boolean=True)
#Defining u, a vector of integers used for eliminating subtours in the optimization.
u = cp.Variable(n, integer=True)
#Creating a column vector ones of size n x 1, used to define constraints.
ones = np.ones((n,1))

The objective function minimizes the total travel distance.
- C[i,j] gives the distance between point i and point j.
- X[i,j] is the decision variable indicating whether the salesperson travels directly from point i to point j.

The product C[i,j] * X[i,j] represents the contribution of that specific connection (if selected) to the total travel distance. The total cost (distance) is computed as the sum of these contributions.

cp.multiply(C, X) performs element-wise multiplication between the distance matrix and the decision variable matrix, and cp.sum() sums these products to compute the total travel distance.

In [5]:
# Defining the objective function(cost function representation)
objective = cp.Minimize(cp.sum(cp.multiply(C, X)))

Constraints
- X @ ones == ones: This constraint ensures that each point i has exactly one outgoing edge, meaning the salesperson leaves every point exactly once.
- X.T @ ones == ones: This ensures that each point j has exactly one incoming edge, meaning the salesperson arrives at every point exactly once.
- cp.diag(X) == 0: This prevents the salesperson from traveling from any point i to itself, i.e., self-loops are not allowed.
- u[1:] >= 2 and u[1:] <= n: These constraints set the range for the integer variables u from 2 to n for all points except the starting point. This ensures a valid sequence order for visiting points.
- u[0] == 1: The order for the starting point (the origin) is fixed as 1, meaning the salesperson starts at this point.
- The constraint u[i] - u[j] + 1 <= (n - 1) * (1 - X[i, j]) ensures that no subtours are formed. If the salesperson travels directly from point i to point j (i.e.,X[i,j]=1), the difference in the order variables u must be consistent with the order of visiting points.
- This constraint prevents closed loops (subtours) that visit only a subset of points and forces the solution to be a single valid tour visiting all points.

In [6]:
# Defining the constraints
constraints = []
#Adding the first constraint: Each point must be visited exactly once (both incoming and outgoing connections must sum to 1 for each point).
constraints += [X @ ones == ones]
constraints += [X.T @ ones == ones]
#Adding the second constraint: The diagonal of matrix X must be zero, ensuring no point is connected to itself.
constraints += [cp.diag(X) == 0]
#The vector u starts at 1 for the first point and ranges between 2 and n for other points.
constraints += [u[1:] >= 2]
constraints += [u[1:] <= n]
constraints += [u[0] == 1]
#Adding the subtour elimination constraint, If X[i, j] == 1 then the difference between u[i] and u[j] must be constrained to eliminate subtours.
for i in range(1, n):
    for j in range(1, n):
        if i != j:
            constraints += [ u[i] - u[j] + 1  <= (n - 1) * (1 - X[i, j]) ]

The convex optimization problem is defined by the objective function (minimizing the total distance) and the constraints (ensuring valid tours and eliminating subtours). The problem is solved using CVXPY’s solver.

In [7]:
# Solving the problem
prob = cp.Problem(objective, constraints)
prob.solve(verbose=False)

22.31000388184043

After solving the problem, the variable X.value contains the optimal tour, represented by a binary matrix indicating the paths chosen. The solution is converted into a human-readable path (sequence of points) by finding the sequence in which points are visited based on the values in X.

In [8]:
# Transforming the solution to a path
# Extracting the solution matrix X and finds the points where X == 1 i.e. indicating a connection between points.
X_sol = np.argwhere(X.value==1)
#Constructing the path by iterating over the solution matrix X_sol to build an ordered list of points representing the optimal tour.
order = X_sol[0].tolist()
for i in range(1, n):
    row = order[-1]
    order.append(X_sol[row,1])

In [10]:
# Showing the optimal path
print('The path is:')
print( ' => '.join(map(str, order)))
# Showing the optimal distance
# Calculateing the total distance of the optimal tour by summing the pairwise distances of connected points.
distance = np.sum(np.multiply(C, X.value))
print('The optimal distance is:', np.round(distance,2), 'km')

The path is:
0 => 4 => 3 => 2 => 1 => 5 => 6 => 0
The optimal distance is: 22.31 km


Summary:
- The Travelling salesman problem inclues multiple linear constraints that need to be satisfied for a valid solution so it is a Constrained Optimizaion Problem
- The objective function involve linear combination of distance matrix C which makes it a Problem of Constrained linear programming 